In [1]:
import pandas as pd
import random


In [2]:
# Cargar el dataset (ajusta la ruta si es necesario)
df = pd.read_csv("dataset/grades.csv")

# Mostrar las primeras filas
df.head()

,StudentID,Parcial1,Parcial2,Parcial3
0,A001,17,10,18
1,A002,10,19,12
2,A003,5,5,13
3,A004,8,6,13
4,A005,16,7,14


In [3]:
def evaluar_offset(df, offset):
    # Aplicar el offset
    df_offset = df.copy()
    parciales = ['Parcial1', 'Parcial2', 'Parcial3']

    # Calcular el promedio con offset
    df_offset['Final'] = df_offset[parciales].mean(axis=1) + offset

    # Porcentaje de aprobados
    aprobados = (df_offset['Final'] >= 11).sum()
    porcentaje_aprobados = aprobados / len(df_offset)

    # Promedio general
    promedio_clase = df_offset['Final'].mean()

    # Penalización: si el promedio general es mayor a 14, fitness es 0
    if promedio_clase > 14:
        return 0
    else:
        return porcentaje_aprobados

In [4]:
def hill_climbing(df, step=0.5, max_iter=100):
    current_offset = 0.0
    best_score = evaluar_offset(df, current_offset)

    for _ in range(max_iter):
        # Generar nuevo offset candidato dentro del rango permitido
        delta = random.choice([-step, step])
        new_offset = current_offset + delta

        # Asegurarse de que esté en [-5, 5]
        if new_offset < -5 or new_offset > 5:
            continue

        new_score = evaluar_offset(df, new_offset)

        # Si mejora, actualizamos
        if new_score > best_score:
            current_offset = new_offset
            best_score = new_score

    return current_offset, best_score


In [5]:
offset_optimo, porcentaje = hill_climbing(df)
print(f"Offset óptimo: {offset_optimo:.2f}")
print(f"Porcentaje de aprobados: {porcentaje * 100:.2f}%")

# Mostrar la nueva distribución de notas
df['Final Ajustado'] = df[['Parcial1', 'Parcial2', 'Parcial3']].mean(axis=1) + offset_optimo
df['Aprobado'] = df['Final Ajustado'] >= 11
df[['StudentID', 'Final Ajustado', 'Aprobado']].head()


Offset óptimo: 2.00
Porcentaje de aprobados: 85.83%


,StudentID,Final Ajustado,Aprobado
0,A001,17.000000,True
1,A002,15.666667,True
2,A003,9.666667,False
3,A004,11.000000,True
4,A005,14.333333,True
